# Minimizing allocations

It seems like [any allocation is deterimental to multithreading](https://discourse.julialang.org/t/poor-performance-on-cluster-multithreading/12248), since garbage collection is single threaded (at least as of 2018). We cannot pre-allocate `M`, but all other intermediate matrices (e.g. Xwork, Hwork, N, temporary vectors) can be preallocated and scaled. 

In [1]:
using Revise
using VCFTools
using MendelImpute
using GeneticVariation
using Random
using SparseArrays
using JLD2, FileIO, JLSO
using ProgressMeter
using GroupSlices
using ThreadPools
using BenchmarkTools
using StatsBase
using StaticArrays
using LinearAlgebra
# using Plots
# using ProfileView

BLAS.set_num_threads(1)

┌ Info: Precompiling MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1273


In [2]:
Threads.nthreads()

8

# Not yet optimized (7/11/2020)

### window by window intersection with global search

In [5]:
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false);

X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 7.59873 seconds
    Computing haplotype pair        = 2.23046 seconds
        BLAS3 mul! to get M and N      = 0.0580204 seconds per thread
        haplopair search               = 1.88659 seconds per thread
        finding redundant happairs     = 0.0253855 seconds per thread
    Phasing by win-win intersection = 0.18561 seconds
    Imputation                      = 6.8954 seconds

 17.168989 seconds (77.19 M allocations: 7.610 GiB, 6.38% gc time)
error_rate = 8.32693826254268e-5


### haplotype thinning

In [6]:
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false, thinning_factor=100, max_haplotypes=100);

X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 7.80156 seconds
    Computing haplotype pair        = 3.30648 seconds
        screening for top haplotypes   = 0.402203 seconds per thread
        BLAS3 mul! to get M and N      = 2.35561 seconds per thread
        haplopair search               = 0.0949087 seconds per thread
        finding redundant happairs     = 0.0348157 seconds per thread
    Phasing by win-win intersection = 0.177686 seconds
    Imputation                      = 6.84377 seconds

 18.479679 seconds (77.72 M allocations: 7.541 GiB, 6.05% gc time)
error_rate = 8.550487693659426e-5


### Lasso

In [10]:
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    lasso = 20, dynamic_programming=false, max_haplotypes=100);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
# X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 7.80041 seconds
    Computing haplotype pair        = 0.683484 seconds
        BLAS3 mul! to get M and N      = 0.104721 seconds per thread
        haplopair search               = 0.307949 seconds per thread
        finding redundant happairs     = 0.0280975 seconds per thread
    Phasing by win-win intersection = 0.179585 seconds
    Imputation                      = 6.49275 seconds

 15.390669 seconds (77.19 M allocations: 7.610 GiB, 7.79% gc time)
error_rate = 8.685062226819259e-5


# Optimized

In [4]:
# global search: Preallocate all vectors/matrices
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false);

X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:07
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 8.25352 seconds
    Computing haplotype pair        = 2.23927 seconds
        BLAS3 mul! to get M and N      = 0.0659687 seconds per thread
        haplopair search               = 1.84551 seconds per thread
        finding redundant happairs     = 0.0299791 seconds per thread
    Phasing by win-win intersection = 0.18616 seconds
    Imputation                      = 6.77303 seconds

 17.844942 seconds (77.19 M allocations: 7.420 GiB, 7.17% gc time)
error_rate = 8.32693826254268e-5


In [3]:
# thinning: preallocate all vectors/matrices
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false, thinning_factor=100, max_haplotypes=100);

X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 7.63071 seconds
    Computing haplotype pair        = 3.21096 seconds
        screening for top haplotypes   = 0.62019 seconds per thread
        BLAS3 mul! to get M and N      = 2.25471 seconds per thread
        haplopair search               = 0.0934914 seconds per thread
        finding redundant happairs     = 0.0637935 seconds per thread
    Phasing by win-win intersection = 0.194177 seconds
    Imputation                      = 6.52209 seconds

 17.627988 seconds (77.20 M allocations: 7.317 GiB, 5.14% gc time)
error_rate = 8.550487693659426e-5


In [10]:
# lasso: preallocate all vectors/matrices
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time hs, ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    lasso = 20, dynamic_programming=false, max_haplotypes=100);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "./compare2/target.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)
rm(outfile, force=true)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:07
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 8.60205 seconds
    Computing haplotype pair        = 3.22841 seconds
        BLAS3 mul! to get M and N      = 0.0361663 seconds per thread
        haplopair search               = 0.180822 seconds per thread
        finding redundant happairs     = 0.0371089 seconds per thread
    Phasing by win-win intersection = 0.216741 seconds
    Imputation                      = 7.02347 seconds

 19.152174 seconds (77.19 M allocations: 7.420 GiB, 6.80% gc time)
error_rate = 8.685062226819259e-5


# Lets do rigorous benchmarks

In [23]:
# first import all data, declare a bunch of (needed or not) variables, and look at 1 window
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"

loaded = JLSO.load(reffile)
compressed_Hunique = loaded[:compressed_Hunique]
X, X_sampleID, X_chr, X_pos, X_ids, X_ref, X_alt = VCFTools.convert_gt(UInt8, tgtfile, trans=true, save_snp_info=true, msg = "Importing genotype file...");

people = size(X, 2)
tgt_snps = size(X, 1)
ref_snps = length(compressed_Hunique.pos)
windows = floor(Int, tgt_snps / width)
num_unique_haps = round(Int, avg_haplotypes_per_window(compressed_Hunique))

ph = [HaplotypeMosaicPair(ref_snps) for i in 1:people]
haplotype1 = [zeros(Int32, windows) for i in 1:people]
haplotype2 = [zeros(Int32, windows) for i in 1:people]

people = size(X, 2)
ref_snps = length(compressed_Hunique.pos)
width = compressed_Hunique.width
windows = length(haplotype1[1])
threads = Threads.nthreads()
inv_sqrt_allele_var = nothing

# working arrys 
happair1 = [ones(Int32, people)           for _ in 1:threads]
happair2 = [ones(Int32, people)           for _ in 1:threads]
hapscore = [zeros(Float32, size(X, 2))    for _ in 1:threads]
Xwork    = [zeros(Float32, width, people) for _ in 1:threads]

# window 1
w = 1
Hw_aligned = compressed_Hunique.CW_typed[w].uniqueH
Xw_idx_start = (w - 1) * width + 1
Xw_idx_end = (w == windows ? length(X_pos) : w * width)
Xw_aligned = view(X, Xw_idx_start:Xw_idx_end, :)
d  = size(Hw_aligned, 2)
id = Threads.threadid()

Importing genotype file...100%|█████████████████████████| Time: 0:00:07


1

In [26]:
# global search must allocate M, Hwork, N for each window, but there's no other allocation
M = zeros(Float32, size(Hw_aligned, 2), size(Hw_aligned, 2))
N = zeros(Float32, size(Xw_aligned, 2), size(Hw_aligned, 2))
Hwork = convert(Matrix{Float32}, Hw_aligned)
@benchmark haplopair!($Xw_aligned, $Hw_aligned, happair1=$(happair1[id]), 
    happair2=$(happair2[id]), hapscore=$(hapscore[id]), Xwork=$(Xwork[id]),
    M=$M, N=$N, Hwork=$Hwork)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     113.290 ms (0.00% GC)
  median time:      116.663 ms (0.00% GC)
  mean time:        117.684 ms (0.00% GC)
  maximum time:     130.187 ms (0.00% GC)
  --------------
  samples:          43
  evals/sample:     1

In [9]:
# thinning must allocate a bunch of vector/matrices. Only R, Hwork cannot be preallocated
# all allocations are due to distance computation.
keep    = 100
maxindx = zeros(Int32, keep)
maxgrad = zeros(Float32, keep)
Xi      = zeros(Float32, size(Xw_aligned, 1))
N       = zeros(Float32, keep)
Hk    = zeros(Float32, size(Hw_aligned, 1), keep)
M     = zeros(Float32, keep, keep)
Xwork = zeros(Float32, size(Xw_aligned, 1), size(Xw_aligned, 2))
Hwork = convert(Matrix{Float32}, Hw_aligned)
R     = rand(Float32, size(Hw_aligned, 2), size(Xw_aligned, 2))
@benchmark MendelImpute.haplopair_thin_BLAS2!($Xw_aligned, $Hw_aligned, allele_freq=nothing, 
    keep=$keep, happair1=$(happair1[id]), happair2=$(happair2[id]), 
    hapscore=$(hapscore[id]), maxindx=$maxindx, maxgrad=$maxgrad, Xi=$Xi, N=$N, 
    Hk=$Hk, M=$M, Xwork=$Xwork, Hwork=$Hwork, R=$R)

BenchmarkTools.Trial: 
  memory estimate:  6.50 KiB
  allocs estimate:  2
  --------------
  minimum time:     163.348 ms (0.00% GC)
  median time:      164.766 ms (0.00% GC)
  mean time:        166.568 ms (0.00% GC)
  maximum time:     191.775 ms (0.00% GC)
  --------------
  samples:          31
  evals/sample:     1

In [10]:
# stepwise: here Hwork, M, Nt cannot be preallocated, but there's no other allocation.
r       = 10
maxindx = zeros(Int32, r)
maxgrad = zeros(Float32, r)
Xwork = zeros(Float32, size(Xw_aligned, 1), size(Xw_aligned, 2))
Hwork = convert(Matrix{Float32}, Hw_aligned)
M     = zeros(Float32, size(Hw_aligned, 2), size(Hw_aligned, 2))
Nt    = zeros(Float32, size(Hw_aligned, 2), size(Xw_aligned, 2))
@benchmark haplopair_stepscreen!($Xw_aligned, $Hw_aligned, r=$r, happair1=$(happair1[id]), 
    happair2=$(happair2[id]), hapscore=$(hapscore[id]), maxindx=$maxindx, 
    maxgrad=$maxgrad, Xwork=$Xwork, Hwork=$Hwork, M=$M, Nt=$Nt)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     27.849 ms (0.00% GC)
  median time:      28.387 ms (0.00% GC)
  mean time:        29.143 ms (0.00% GC)
  maximum time:     38.266 ms (0.00% GC)
  --------------
  samples:          172
  evals/sample:     1

In [11]:
# test saving result step
function test(haplotype1, haplotype2, happair1, happair2, id, w, people, compressed_Hunique)
    @inbounds for i in 1:people
        haplotype1[i][w] = MendelImpute.unique_idx_to_complete_idx(
            happair1[id][i], w, compressed_Hunique)
        haplotype2[i][w] = MendelImpute.unique_idx_to_complete_idx(
            happair2[id][i], w, compressed_Hunique)
    end
end
@btime test($haplotype1, $haplotype2, $happair1, $happair2, $id, 
    $w, $people, $compressed_Hunique)

  5.412 μs (0 allocations: 0 bytes)


# Try manually unrolling

In [2]:
# first import all data, declare a bunch of (needed or not) variables, and look at 1 window
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"

loaded = JLSO.load(reffile)
compressed_Hunique = loaded[:compressed_Hunique]
X, X_sampleID, X_chr, X_pos, X_ids, X_ref, X_alt = VCFTools.convert_gt(UInt8, tgtfile, trans=true, save_snp_info=true, msg = "Importing genotype file...");

people = size(X, 2)
tgt_snps = size(X, 1)
ref_snps = length(compressed_Hunique.pos)
windows = floor(Int, tgt_snps / width)
num_unique_haps = round(Int, avg_haplotypes_per_window(compressed_Hunique))

ph = [HaplotypeMosaicPair(ref_snps) for i in 1:people]
haplotype1 = [zeros(Int32, windows) for i in 1:people]
haplotype2 = [zeros(Int32, windows) for i in 1:people]

people = size(X, 2)
ref_snps = length(compressed_Hunique.pos)
width = compressed_Hunique.width
windows = length(haplotype1[1])
threads = Threads.nthreads()
inv_sqrt_allele_var = nothing

# working arrys 
happair1 = [ones(Int32, people)           for _ in 1:threads]
happair2 = [ones(Int32, people)           for _ in 1:threads]
hapscore = [zeros(Float32, size(X, 2))    for _ in 1:threads]
Xwork    = [zeros(Float32, width, people) for _ in 1:threads]

# window 1
w = 1
Hw_aligned = compressed_Hunique.CW_typed[w].uniqueH
Xw_idx_start = (w - 1) * width + 1
Xw_idx_end = (w == windows ? length(X_pos) : w * width)
Xw_aligned = view(X, Xw_idx_start:Xw_idx_end, :)
d  = size(Hw_aligned, 2)
id = Threads.threadid()

Importing genotype file...100%|█████████████████████████| Time: 0:00:07


1

In [8]:
# global search: unrolling seems 15% faster
M = zeros(Float32, size(Hw_aligned, 2), size(Hw_aligned, 2))
N = zeros(Float32, size(Xw_aligned, 2), size(Hw_aligned, 2))
Hwork = convert(Matrix{Float32}, Hw_aligned)
@benchmark haplopair!($Xw_aligned, $Hw_aligned, happair1=$(happair1[id]), 
    happair2=$(happair2[id]), hapscore=$(hapscore[id]), Xwork=$(Xwork[id]),
    M=$M, N=$N, Hwork=$Hwork)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     100.562 ms (0.00% GC)
  median time:      102.703 ms (0.00% GC)
  mean time:        103.065 ms (0.00% GC)
  maximum time:     112.184 ms (0.00% GC)
  --------------
  samples:          49
  evals/sample:     1

# Try StaticArrays

In [15]:
# first import all data, declare a bunch of (needed or not) variables, and look at 1 window
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"

loaded = JLSO.load(reffile)
compressed_Hunique = loaded[:compressed_Hunique]
X, X_sampleID, X_chr, X_pos, X_ids, X_ref, X_alt = VCFTools.convert_gt(UInt8, tgtfile, trans=true, save_snp_info=true, msg = "Importing genotype file...");

people = size(X, 2)
tgt_snps = size(X, 1)
ref_snps = length(compressed_Hunique.pos)
windows = floor(Int, tgt_snps / width)
num_unique_haps = round(Int, avg_haplotypes_per_window(compressed_Hunique))

ph = [HaplotypeMosaicPair(ref_snps) for i in 1:people]
haplotype1 = [zeros(Int32, windows) for i in 1:people]
haplotype2 = [zeros(Int32, windows) for i in 1:people]

people = size(X, 2)
ref_snps = length(compressed_Hunique.pos)
width = compressed_Hunique.width
windows = length(haplotype1[1])
threads = Threads.nthreads()
inv_sqrt_allele_var = nothing

# working arrys 
happair1 = [SizedVector{people}(ones(Int32, people))            for _ in 1:threads]
happair2 = [SizedVector{people}(ones(Int32, people))            for _ in 1:threads]
hapscore = [SizedVector{size(X, 2)}(zeros(Float32, size(X, 2))) for _ in 1:threads]
Xwork    = [zeros(Float32, width, people) for _ in 1:threads]

# window 1
w = 1
Hw_aligned = compressed_Hunique.CW_typed[w].uniqueH
Xw_idx_start = (w - 1) * width + 1
Xw_idx_end = (w == windows ? length(X_pos) : w * width)
Xw_aligned = view(X, Xw_idx_start:Xw_idx_end, :)
d  = size(Hw_aligned, 2)
id = Threads.threadid()

Importing genotype file...100%|█████████████████████████| Time: 0:00:07


1

In [22]:
# only used SizedArray for vectors
M = zeros(Float32, size(Hw_aligned, 2), size(Hw_aligned, 2))
N = zeros(Float32, size(Xw_aligned, 2), size(Hw_aligned, 2))
Hwork = convert(Matrix{Float32}, Hw_aligned)
@benchmark haplopair!($Xw_aligned, $Hw_aligned, happair1=$(happair1[id]), 
    happair2=$(happair2[id]), hapscore=$(hapscore[id]), Xwork=$(Xwork[id]),
    M=$M, N=$N, Hwork=$Hwork)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     118.157 ms (0.00% GC)
  median time:      121.635 ms (0.00% GC)
  mean time:        122.696 ms (0.00% GC)
  maximum time:     132.345 ms (0.00% GC)
  --------------
  samples:          41
  evals/sample:     1

In [20]:
# used SizedArray for M and N too
n = size(Xw_aligned, 2)
d = size(Hw_aligned, 2)
M = SizedMatrix{d, d}(zeros(Float32, d, d))
N = SizedMatrix{n, d}(zeros(Float32, n, d))
Hwork = convert(Matrix{Float32}, Hw_aligned)
@benchmark haplopair!($Xw_aligned, $Hw_aligned, happair1=$(happair1[id]), 
    happair2=$(happair2[id]), hapscore=$(hapscore[id]), Xwork=$(Xwork[id]),
    M=$M, N=$N, Hwork=$Hwork)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     524.865 ms (0.00% GC)
  median time:      528.679 ms (0.00% GC)
  mean time:        530.168 ms (0.00% GC)
  maximum time:     540.517 ms (0.00% GC)
  --------------
  samples:          10
  evals/sample:     1

# compute_optimal_haplotypes!

In [2]:
# first import all data, declare a bunch of (needed or not) variables, and look at 1 window
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"

loaded = JLSO.load(reffile)
compressed_Hunique = loaded[:compressed_Hunique]
X, X_sampleID, X_chr, X_pos, X_ids, X_ref, X_alt = VCFTools.convert_gt(UInt8, tgtfile, trans=true, save_snp_info=true, msg = "Importing genotype file...");

people = size(X, 2)
tgt_snps = size(X, 1)
ref_snps = length(compressed_Hunique.pos)
windows = floor(Int, tgt_snps / width)
num_unique_haps = round(Int, avg_haplotypes_per_window(compressed_Hunique))

ph = [HaplotypeMosaicPair(ref_snps) for i in 1:people]
haplotype1 = [zeros(Int, windows) for i in 1:people]
haplotype2 = [zeros(Int, windows) for i in 1:people]

# working arrys 
timers = [zeros(5*8) for _ in 1:Threads.nthreads()] # 8 for spacing
pmeter = Progress(windows, 5, "Computing optimal haplotypes...")
happair1 = [ones(Int, people)           for _ in 1:Threads.nthreads()]
happair2 = [ones(Int, people)           for _ in 1:Threads.nthreads()]
hapscore = [zeros(Float32, size(X, 2))    for _ in 1:Threads.nthreads()]
Xwork    = [zeros(Float32, width, people) for _ in 1:Threads.nthreads()];

Importing genotype file...100%|█████████████████████████| Time: 0:00:07


In [7]:
@time MendelImpute.compute_optimal_haplotypes!(haplotype1, haplotype2, compressed_Hunique, 
    X, X_pos, nothing, nothing, false, 800, false, timers, pmeter, happair1, happair2, 
    hapscore, Xwork);

 14.759730 seconds (806 allocations: 371.056 MiB, 0.22% gc time)


In [10]:
@btime MendelImpute.compute_optimal_haplotypes!($haplotype1, $haplotype2, $compressed_Hunique, 
    $X, $X_pos, nothing, nothing, false, 800, false, $timers, $pmeter, $happair1, $happair2, 
    $hapscore, $Xwork);

  13.437 s (802 allocations: 371.06 MiB)


# Writing routine

In [10]:
# create variables
Random.seed!(2020)
width   = 512
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.w$width.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, width = width,
    dynamic_programming = false);

loaded = JLSO.load(reffile)
compressed_Hunique = loaded[:compressed_Hunique]

X, X_sampleID, X_chr, X_pos, X_ids, X_ref, X_alt = VCFTools.convert_gt(UInt8, tgtfile, trans=true, save_snp_info=true, msg = "Importing genotype file...")
XtoH_idx = indexin(X_pos, compressed_Hunique.pos) # X_pos[i] == H_pos[XtoH_idx[i]]
X_full = Matrix{Union{Missing, UInt8}}(missing, length(compressed_Hunique.pos), size(X, 2))
copyto!(@view(X_full[XtoH_idx, :]), X); # keep known entries
impute_discard_phase!(X_full, compressed_Hunique, ph)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:07
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 72, averaging ~ 627 unique haplotypes per window.

Timings: 
    Data import                     = 8.4725 seconds
    Computing haplotype pair        = 3.33952 seconds
        BLAS3 mul! to get M and N      = 0.0983789 seconds per thread
        haplopair search               = 2.90371 seconds per thread
        finding redundant happairs     = 0.0345083 seconds per thread
    Phasing by win-win intersection = 0.287375 seconds
    Imputation                      = 7.33042 seconds

 19.804090 seconds (78.25 M allocations: 7.471 GiB, 5.16% gc time)


Importing genotype file...100%|█████████████████████████| Time: 0:00:07


In [ ]:
@time write(outfile, X_full, compressed_Hunique, X_sampleID)

In [18]:
# original
@btime write($outfile, $X_full, $compressed_Hunique, $X_sampleID) seconds=30

Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:05
Writing to file...100%|█████████████████████████████████| Time: 0:00:05
Writing to file...100%|█████████████████████████████████| Time: 0:00:05
Writing to file...100%|█████████████████████████████████| Time: 0:00:05
Writing to file...100%|█████████████████████████████████| Time: 0:00:05
Writing to file...100%|█████████████████████████████████| Time: 0:00:05
Writing to file...100%|█████████████████████████████████| Time: 0:00:05
Writing to file...100%|█████████████████████████████████| Time: 0:00:05
Writing to file...100%|█████████████████████████████████| Time: 0:00:05
Writing to file...100%|█████████████████████████████████| Time: 0:00:05
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 

  5.933 s (414103 allocations: 362.05 MiB)


In [16]:
# copy
@btime write($outfile, $X_full, $compressed_Hunique, $X_sampleID) seconds=30

Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


  6.211 s (414104 allocations: 362.05 MiB)
